# Solución CODERHOUSE Data Engineering Pre Entrega 1

## Libraries

In [1]:
import os

In [2]:
import requests

In [3]:
import pandas as pd

In [4]:
import psycopg2
from psycopg2 import sql

In [5]:
from dotenv import load_dotenv

## Cargar valores de configuración

In [6]:
load_dotenv()

True

## Classes

In [7]:
class APIHandler:
    def get_repos(self, topic: str ="llm") -> pd.DataFrame:
        """Get github repos that contain a topic value."""
        page = 1
        per_page = 20
        uri = "api.github.com/search/repositories"
        url = f"https://{uri}?q={topic}+in:name&page={page}&per_page={per_page}"
        headers = {
            "Accept": "application/vnd.github.v3+json"
        }
        
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            raise ValueError(f"Error fetching data: {response.status_code}")
    
        keys = [
            "id",
            "name",
            "description",
            "created_at",
            "updated_at",
            "html_url",
        ]
        repos = response.json()["items"]
        repos =  [{key: d[key] for key in keys} for d in repos]      
    
        return pd.DataFrame(repos)

In [8]:
class DBHandler:
    def __init__(self) -> None:
        self.engine = None

    def _connect(self) -> None:
        if self.engine:
            return

        host = os.environ["DB_HOST"]
        port = os.environ["DB_PORT"]
        database = os.environ["DB_NAME"]
        username = os.environ["DB_USERNAME"]
        password = os.environ["DB_PASSWORD"]

        self.engine = psycopg2.connect(
            host=host,
            dbname=database,
            user=username,
            password=password,
            port=port
        )
        
    def _disconnect(self) -> None:
        if not self.engine:
            return

        self.engine.close()
    
    def populate(self, df: pd.DataFrame) -> None:
        self._connect()
        self._create_table()
        with self.engine.cursor() as cur:
            cur.execute("SELECT id FROM repos")
            records = cur.fetchall()
            ids = [record[0] for record in records]
            df_new = df[~df["id"].isin(ids)]

            if not df_new.empty:                           
                data_tuples = [tuple(x) for x in df_new.to_numpy()]
                insert_query = sql.SQL("INSERT INTO repos (id, name, description, created_at, updated_at, html_url) VALUES {}").format(
                    sql.SQL(',').join(map(sql.Literal, data_tuples))
                )
                cur.execute(insert_query)         
                self.engine.commit()
            
        self._disconnect()

    def _create_table(self):
        with self.engine.cursor() as cur:
            cur.execute("""
                CREATE TABLE IF NOT EXISTS repos (
                    id INTEGER PRIMARY KEY,
                    name VARCHAR(500),
                    description VARCHAR(500) NULL,
                    created_at TIMESTAMP,
                    updated_at TIMESTAMP,
                    html_url VARCHAR(500)
                );
            """)
            self.engine.commit()        

## Main

In [9]:
api_handler = APIHandler()

In [10]:
repos = api_handler.get_repos()

In [11]:
repos.sample(5)

,id,name,description,created_at,updated_at,html_url
4,634081686,mlc-llm,"Enable everyone to develop, optimize and deplo...",2023-04-29T01:59:25Z,2024-01-26T20:38:39Z,https://github.com/mlc-ai/mlc-llm
11,627561173,web-llm,Bringing large-language models and chat to web...,2023-04-13T18:11:59Z,2024-01-27T01:53:27Z,https://github.com/mlc-ai/web-llm
15,634046345,llm-foundry,LLM training code for MosaicML foundation models,2023-04-28T22:33:27Z,2024-01-26T18:26:53Z,https://github.com/mosaicml/llm-foundry
3,655099582,llm-course,Course to get into Large Language Models (LLMs...,2023-06-17T22:16:25Z,2024-01-27T01:55:52Z,https://github.com/mlabonne/llm-course
16,643916827,Awesome-Chinese-LLM,整理开源的中文大语言模型，以规模较小、可私有化部署、训练成本较低的模型为主，包括底座模型，垂...,2023-05-22T12:27:03Z,2024-01-26T18:18:22Z,https://github.com/HqWu-HITCS/Awesome-Chinese-LLM


In [12]:
db_handler = DBHandler()

In [13]:
db_handler.populate(repos)

SyntaxError: syntax error at or near "ON"
LINE 1: ...Z', 'https://github.com/liguodongiot/llm-action') ON CONFLIC...
                                                             ^
